In [ ]:
from arcgis.gis import GIS
from getpass import getpass

import pandas as pd
import logging
from datetime import datetime
import sys, os
import tempfile
import traceback
import re
import openpyxl

In [ ]:
# Source Portal information
source_admin_username = 'adminuser'
source_portal_url = 'https://sourceorg.maps.arcgis.com'
#source_password = getpass(prompt='Please enter the password for the source Portal') # This will prompt you for the password rather then storing it
source_password = 'password'

# Target Portal information
target_admin_username = 'adminuser'
target_portal_url = 'https://targetorg.arcgis.com'
#target_password = getpass(prompt='Please enter the password for the target Portal') # This will prompt you for the password rather then storing it
target_password = 'password'

org = 'newOrg'

# Log file location - specify the location of the log file to be created
logging.basicConfig(filename = r".\CopyUsers_log.txt", level=logging.INFO)
now = datetime.datetime.now()
logging.info("{}  Begin user migration".format(str(now)))

basePath = r"."
usermapXLS = os.path.join(basePath, "User_Mapping.xlsx")

In [ ]:
# Instantiate Portal connections - use verify_cert = False to use self signed SSL
source = GIS(source_portal_url, source_admin_username, source_password, verify_cert = False, expiration = 9999)
logging.info("Connected to source portal "+source_portal_url+" as "+source_admin_username)
target = GIS(target_portal_url, target_admin_username, target_password, verify_cert = False)
logging.info("Connected to target portal "+target_portal_url+" as "+target_admin_username)

In [ ]:
# Get source users source_users = source.users.search('!esri_ & !admin',max_users=99999)
source_users = source.users.search('!esri_ & !esri_livingatlas',max_users=99999)

print (source_users)
logging.info("Total users to migrate: {}".foramt(len(source_users)))

In [ ]:
def GetUserTypeName(userType):
    userTypes = [{"Type": "creatorUT","Descr": "Creator"}, {"Type": "viewerUT","Descr": "Viewer"}]
    typeDescr = next(item for item in userTypes if item["Type"]==userType)
    return typeDescr["Descr"]

In [ ]:

def copy_user(target_portal, source_user, password, org):
    # split the fullName
    full_name = source_user.fullName
    first_name = full_name.split()[0]
    try:
        last_name = full_name.split(maxsplit = 1)[1]
    except:
        last_name = 'NoLastName'
    
    newusername = source_user.username
    if re.search(org, newusername, re.IGNORECASE):
        newusername = "{}_{}".format(newusername, "1")
    else:
        newusername = "{}_{}".format(newusername, org)
        
    userRole = source_user.role
    if userRole == "org_admin":
        userRole = "org_user"

    try: 
        target_user = target_portal.users.create(username = newusername,
                                                 password = password, 
                                                 firstname = first_name, 
                                                 lastname = last_name, 
                                                 email = source_user.email, 
                                                 description = source_user.description, 
                                                 role = userRole,
                                                 user_type = "Creator")

        if source_user.role == "org_admin":
            target_user.update_role("org_admin")
        
        # update user properties
        target_user.update(access = source_user.access, 
                           description = source_user.description, 
                           tags = source_user.tags)

        return target_user
    
    except Exception as Ex:
        print(str(Ex))
        print("Unable to create user "+ newusername)
        print (str(sys.exc_info()) + "\n")
        print(traceback.format_tb(sys.exc_info()[2])[0] + "\n")
        return None

In [ ]:
#  Cycle through source users and run copy routine
userMapping = []
for u in source_users:
    print(u.username)
    userMap = {}
    userMap["sourcename"] = u.username
    userMap["fullname"] = u.fullName
    
    targetUserCheck = target.users.get("{}_{}".format(u.username, org))
    if targetUserCheck:
        print ("Username {} already in target site".format(u.username))
        userMap["targetname"] = "Already in Target"
        continue

    new_user = copy_user(target, u, "ChangeMe12345", org)
    if new_user:
        userMap["targetname"] = new_user.username
    else:
        userMap["targetname"] = "Failed to Copy"
        
    userMapping.append(userMap)

In [ ]:
userMapping

In [ ]:
#Export User Mapping to XLS sheet
df = pd.DataFrame.from_dict(userMapping)
with pd.ExcelWriter(usermapXLS, engine='openpyxl') as writer:
    df.to_excel(writer, sheet_name='Sheet1', index=None)

## Copy one user

In [ ]:
#Failed to copy any over?
users = ["e000050"]
for user in users:
    u = source.users.get(user)
    new_user = copy_user(target, u, "ChangeMe12345", org)
    print (new_user.username)